In [2]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pyarrow.parquet as pq
import pandas as pd
from rich import print
import warnings
import datetime
from astropy.time import Time
import os

ModuleNotFoundError: No module named 'pyarrow'

In [2]:
def counts2mag(cps, band):
    scale = 18.82 if band == 'FUV' else 20.08
    with np.errstate(invalid='ignore'):
        with warnings.catch_warnings():
            warnings.simplefilter("ignore")
            mag = -2.5 * np.log10(cps) + scale
    return mag

In [3]:
def angularSeparation(ra1, dec1, ra2, dec2):

    d2r = np.pi/180.
    ra2deg = 1./d2r

    d1 = dec1*d2r
    d2 = dec2*d2r

    r1 = ra1*d2r
    r2 = ra2*d2r

    a = np.sin((d2-d1)/2.)**2.+np.cos(d1)*np.cos(d2)*np.sin((r2-r1)/2.)**2.
    r = 2*np.arcsin(np.sqrt(a))

    return r*ra2deg

In [4]:
header_data = pd.read_csv('../ref/mislike_image_header_table.csv')
catalog_filename = '../ref/catalog_nd_daostarfinder.parquet'
catalog_file = pq.ParquetFile(catalog_filename)

In [5]:
def make_target_table(objid_list,morphology=''):
    table = pd.DataFrame()
    for objid in objid_list:
        eclipse = int(str(objid)[-5:])
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',objid)]).to_pandas()
        this_star['morphology'] = morphology
        table = pd.concat([table,this_star[["eclipse","obj_id","ra","dec","morphology"]]])
    return table

In [6]:
gfcat_corpus = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_gifs/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_corpus = pd.concat([gfcat_corpus,table])
print(f'{len(gfcat_corpus)} sources found in general search')

gfcat_targeted = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_known_jpg/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_targeted = pd.concat([gfcat_targeted,table])
print(f'{len(gfcat_targeted)} sources found in targeted search')

gfcat_fuv_matched = pd.DataFrame()
path = "/Users/cm/GFCAT/gfcat/test_fuv_matched/sorted/"
for d in ['flare','eclipse','trend','unk variable']:
    targets = []
    for fn in os.listdir(f"{path}{d}/"):
        try:
            targets+=[int(fn.split('-')[1])]
        except IndexError:
            continue
    table = make_target_table(targets,morphology=d.upper()[0])
    gfcat_fuv_matched = pd.concat([gfcat_fuv_matched,table])
print(f'{len(gfcat_fuv_matched)} sources found in matched FUV targets')



865 sources found in general search

654 sources found in targeted search

281 sources found in matched FUV targets

In [7]:
def make_variable_target_table(obj_ids):
    variable_table = pd.DataFrame()
    for obj_id in obj_ids:
        eclipse = int(str(obj_id)[-5:])
        #obj_id = int(objid)
        this_star = pq.read_table(catalog_filename,filters =
                                  [('eclipse','=',eclipse),
                                   ('obj_id','=',obj_id)]).to_pandas()
        expt = header_data.loc[header_data['ECLIPSE']==eclipse].loc[header_data['BAND']=='NUV']['EXPT_0']
        cps = this_star["aperture_sum_n_12_8"].tolist()[0]/expt.tolist()[0]
        cps_err = np.sqrt(this_star["aperture_sum_n_12_8"].tolist()[0])/expt.tolist()[0]
        mag = counts2mag(cps,'NUV')
        mag_err_1 = mag-counts2mag(cps+cps_err,'NUV')
        mag_err_2 = counts2mag(cps-cps_err,'NUV')-mag # this one is always larger
        this_star['cps'] = cps
        this_star['cps_err'] = cps_err
        this_star['NUVmag'] = mag
        this_star['NUVmag_err_1'] = mag_err_1
        this_star['NUVmag_err_2'] = mag_err_2
        variable_table = pd.concat([variable_table,
                                 this_star[
                                    ["obj_id","ra","dec","eclipse","NUVmag","NUVmag_err_1","NUVmag_err_2",]]])
    return variable_table


In [8]:
# define manually adjusted source positions
mpt = """06882 99999906882 310.03898  -0.86965 F 
15112 99999915112 173.68275  45.53918 F
29166 99999929166 355.72871   1.65753 F
29390 99999929390  10.92245 -42.34763 U
29643 99999929643  53.06506 -29.51007 U
43201 99999943201 217.15856  57.75332 U
03858 99999903858 135.96049  52.35710 F
04028 99999904028 189.00277  62.49956 F
07940 99999907940   6.43415  17.08139 F
09709 99999909709 164.46273  -3.76613 U
13533 99999913533  16.65372  45.38989 F
13656 99999913656  24.76303 -17.94913 F
16497 99999916497 219.53099  33.10083 F
18081 99999918081 358.46939 -10.54433 F
18380 99999918380   7.70821 -71.84528 F
18753 99999918753  51.26207 -17.21407 F
23505 99999923505  13.11492   0.09855 F
26488 99999926488 212.80185   6.83090 F
28619 99999928619 326.15289   0.70565 F
30122 99999930122  53.00912 -27.04739 F
30668 99999930668 133.73297   2.57409 F
31274 99999931274 120.70829  36.45803 F
36705 99999936705 146.54298  15.54300 F
40521 99999940521  29.19705  -0.35408 F
40930 99999940930  70.63517  18.43965 F
42499 99999942499 165.10157  12.73207 F
43238 99999943238 243.41123  54.66503 F
43828 99999943828 338.34099 -60.93026 F
44684 99999944684 308.13204   7.97132 F
45629 99999945629 160.64052  76.02885 F"""

gfcat_adjusted = pd.DataFrame(
    {'eclipse':np.array(np.array([t.split() for t in mpt.split('\n')])[:,0],dtype='int'),
     'obj_id':np.array(np.array([t.split() for t in mpt.split('\n')])[:,1],dtype='int64'),
     'ra':np.array(np.array([t.split() for t in mpt.split('\n')])[:,2],dtype='float'),
     'dec':np.array(np.array([t.split() for t in mpt.split('\n')])[:,3],dtype='float'),
     'morphology':np.array(np.array([t.split() for t in mpt.split('\n')])[:,4],dtype='str')})

In [9]:
# compile both targeted and untargeted sources
#gfcat_draft = pd.concat([make_variable_target_table(gfcat_corpus['obj_id'].values),
#                         make_variable_target_table(gfcat_targeted['obj_id'].values),
#                         make_variable_target_table(gfcat_fuv_matched['obj_id'].values),
#                         gfcat_adjusted])
#print(len(gfcat_draft))

In [17]:
gfcat_old_finds = pd.read_csv('missing_gfcat_visits_221218.csv',index_col=None)
gfcat_old_finds.iloc[-12:]

,eclipse,obj_id,ra,dec,morphology
534,15133,9.999992e+10,204.17073,51.91460,E
535,4276,9.999990e+10,149.84515,2.96435,F
536,24237,9.999992e+10,358.90042,0.69566,F
537,21750,9.999992e+10,202.73628,65.71721,U
538,9939,9.999991e+10,215.08597,52.26377,E
539,7471,9.999991e+10,346.70626,-0.61413,T
540,7545,9.999991e+10,21.95165,-28.89494,F
541,30574,9.999993e+10,127.88794,32.46371,U
542,28449,9.999993e+10,351.76689,7.36030,T
543,15251,9.999992e+10,194.85851,27.35632,U


In [38]:
gfcat_combined = pd.concat([gfcat_corpus,gfcat_targeted,gfcat_adjusted,gfcat_fuv_matched,
                            gfcat_old_finds.iloc[-12:]]) # just the repointed ones
print(f"Starting with {len(gfcat_combined)} identified variable source-visits")
gfcat_combined.to_csv('/Users/cm/Downloads/temp.csv')

Starting with 1842 identified variable source-visits

In [34]:
un,ix=np.unique(gfcat_combined['obj_id'],return_index=True)
gfcat_visits = gfcat_combined.reset_index()
gfcat_visits

,index,eclipse,obj_id,ra,dec,morphology
0,0,1422,1.479501e+09,343.352654,-39.793072,F
1,0,12992,1.301613e+09,352.094400,0.477522,F
2,0,1640,1.297002e+09,260.307220,58.102305,F
3,0,15994,2.368216e+09,195.957862,-17.314252,F
4,0,28450,1.340328e+09,333.819312,1.061756,F
...,...,...,...,...,...,...
1837,541,30574,9.999993e+10,127.887940,32.463710,U
1838,542,28449,9.999993e+10,351.766890,7.360300,T
1839,543,15251,9.999992e+10,194.858510,27.356320,U
1840,544,23279,9.999992e+10,4.654980,16.341350,F


In [90]:
gfcat_visits = gfcat_combined.sort_values('eclipse').reset_index()[['eclipse','obj_id','ra','dec','morphology']]
gfcat_visits
for e in np.unique(gfcat_combined['eclipse']):
    visits = gfcat_visits[gfcat_visits['eclipse']==e]
    # if there is only one source in the eclipse, move on
    if len(visits)==1:
        continue
    d = angularSeparation(visits['ra'].values.mean(),
                          visits['dec'].values.mean(),
                          visits['ra'].values,visits['dec'].values)
    # only keep one of each unique pointing in the eclipse
    d_all = np.unique(d)
    for d_u in d_all:
        ix = np.where(d==d_u)
        if len(ix[0])==1:
            continue
        gfcat_visits.drop(visits.iloc[ix].index[1:],inplace=True)
        
    visits = gfcat_visits[gfcat_visits['eclipse']==e]
    # if there is now only one source in the eclipse, move on
    if len(visits)==1:
        continue
        
    d = angularSeparation(visits['ra'].values.mean(),
                          visits['dec'].values.mean(),
                          visits['ra'].values,visits['dec'].values)
    if len(visits)==2:
        # there are only two objects and they are far apart, so separate
        if all(d>35/60/60):
            continue
        # if the object is repointed, use the repointed one
        adj_test = [obj_id.startswith('999999') for obj_id in visits['obj_id'].astype('str').values]
        if any(adj_test):
            for v in visits.iterrows():
                if str(v[1]['obj_id']).startswith('999999'):
                    pass
                else:
                    gfcat_visits.drop(v[0],inplace=True)
            continue
    # manual confirmed to be different sources
    if e in [7500,15895,23102,30868]:
        continue
    elif e==23279:
        gfcat_visits.drop(1098,inplace=True)
        continue
    print(d)
    print(gfcat_visits[gfcat_visits['eclipse']==e])
gfcat_visits = pd.concat([gfcat_visits,gfcat_old_finds.iloc[:-12]])

[0.00379558 0.00379559]

eclipse        obj_id          ra       dec morphology
291     7784  1.243108e+09  352.933228 -2.750752          F
292     7784  1.248908e+09  352.937941 -2.744797          F

[0.00137792 0.00137792]

eclipse       obj_id          ra        dec morphology
414     9625  656109625.0  127.906110  19.396568          F
415     9625  657809625.0  127.906743  19.393878          F

[0.001283 0.001283]

...

eclipse        obj_id          ra        dec morphology
1062    22274  2.833422e+09  249.707971  40.914823          F
1063    22274  2.838122e+09  249.710640  40.913001          F

[0.00227928 0.00227928]

eclipse       obj_id          ra        dec morphology
1251    27109  825627109.0  259.975935  26.501314          F
1252    27109  814227109.0  259.970842  26.501225          F

[0.00133431 0.00133431]

eclipse       obj_id          ra        dec morphology
1546    35599  743035599.0  127.905891  19.396187          F
1547    35599  744235599.0  127.906492  19.393580          F

In [95]:
# This is the observation of Ross 867 & 868 that flare within a few minutes of each other
# Overzealous crossmatching would merge this. So ensure that it does not!
assert len(gfcat_visits[gfcat_visits['eclipse']==27109])==2

In [100]:
gfcat_visits.reset_index()[['eclipse','obj_id','ra','dec','morphology']].to_csv(
    'gfcat_visit_table_positions.csv')#,index=None)

,index,eclipse,obj_id,ra,dec,morphology
0,0,927,3.715901e+09,249.909813,41.112522,U
1,1,1413,1.953301e+09,311.565113,-4.930670,F
2,2,1420,1.468701e+09,315.935391,-7.379244,F
3,3,1422,1.479501e+09,343.352654,-39.793072,F
4,4,1534,1.091702e+09,259.445075,59.689607,F
...,...,...,...,...,...,...
2060,529,46807,NaN,183.176317,-26.926904,U
2061,530,46818,NaN,184.069175,-30.364380,U
2062,531,46828,NaN,183.706376,-32.431746,F
2063,532,46828,NaN,183.710681,-32.431776,F
